# A CNN-LSTM framework for the solar wind density forecasting
## ConvLSTM training
In this notebook we train a ConvLSTM network to predict solar wind densities (electrons + protons)


#### Notebook Contributors
* Andrea Giuseppe Di Francesco -- email: difrancesco.1836928@studenti.uniroma1.it
* Massimo Coppotelli -- email: coppotelli.1705325@studenti.uniroma1.it

In [1]:
# !pip install pandas
# !pip install numpy
# !pip install torch
# !pip install matplotlib
# !pip install torchvision
# !pip install wandb
# !pip3 install pytorch-lightning==1.5.10


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import wandb
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import Callback

# Personal files
from convlstm import *
from cnn_lstm import *

from utils import *
from init import *

In [2]:

if wb:
    wandb.login()

SEED = 312023

pl.seed_everything(SEED)

wandb: Currently logged in as: difra00. Use `wandb login --relogin` to force relogin
Global seed set to 312023


312023

In [3]:
wind_dataset = pd.read_csv('./datasets/wind_dataset_0.5d_res.csv', index_col = 0)

## Pytorch lightning code 
- Need of a collate function to preprocess data: sun_images are expressed as lists in a json files, thus we need a preprocessing before feed them into the ConvLSTM.
- Then we define a Lightning DataModule, and finally the Lightining Module for the model training.

In [4]:
class pl_Dataset_(pl.LightningDataModule):

    def __init__(self,  dataset, bs):
      

      self.train_set = dataset.loc[0:round(len(dataset)*train_split)]
      self.val_set = dataset.loc[round(len(dataset)*train_split)+1: round(len(dataset)*train_split) + round(len(dataset)*val_split)]

      self.bs = bs

    def setup(self, stage = None):
        if stage == 'fit':
            self.train_dataset = DataSet(self.train_set)
        elif stage == 'test':
            self.val_dataset = DataSet(self.val_set)
            

    def train_dataloader(self, *args, **kwargs):
        return DataLoader(self.train_dataset, batch_size = self.bs, shuffle = False, collate_fn = collate) #, transform = transform)  Transformation was already made during the processing

    def val_dataloader(self, *args, **kwargs):
        return DataLoader(self.val_dataset, batch_size = self.bs, shuffle = False, collate_fn = collate) #, transform = transform)


In [5]:
SettingData = pl_Dataset_(wind_dataset, batch_size)

SettingData.setup('fit')
SettingData.setup('test')

if mod == 'conv':
    hyperparameters = training_hp
elif mod == 'cnnlstm':
    hyperparameters = training_hp2

In [6]:
class Save_Model_(Callback):

  ''' This Callback is fundamental to save the model, and also its performances over time. It considers also if we want to save the model, given a path, or if we do not want to do so. '''
  def __init__(self, path, experiment_name):
    self.path = path
    self.exp_name = experiment_name

  def on_train_epoch_end(self, trainer, pl_module):
    model_weights = pl_module.model.state_dict()
    #self.path = self.path + self.exp_name +'.pt'
    save_model({'model_state':model_weights}, self.path)

class Get_Metrics(Callback):

  def __init__(self):
    r = 4
 
  def on_train_epoch_end(self, trainer: "pl.Trainer", pl_module: "pl.LightningModule"):

    mean_train_loss = sum(pl_module.loss_train)/len(pl_module.loss_train)
    
    mean_test_loss = sum(pl_module.loss_test)/len(pl_module.loss_test)

    pl_module.log(name = 'Loss on train', value = mean_train_loss)
    pl_module.log(name = 'Loss on test', value = mean_test_loss)
    R_prt = torch.corrcoef(pl_module.R_prt)[0,1].item()
    R_elc = torch.corrcoef(pl_module.R_elc)[0,1].item()
    pl_module.log(name = 'Electron Density correlation (R)', value = R_elc)
    pl_module.log(name = 'Proton Density correlation (R)', value = R_prt)


    pl_module.loss_train = []
    pl_module.loss_test = []
    pl_module.R_elc = torch.tensor([[]])  
    pl_module.R_prt = torch.tensor([[]])




In [7]:
class TrainingModule(pl.LightningModule):

    def __init__(self, model, experiment_name):
        super().__init__()
        self.model = model #HeliosNet(n_channels, n_hidden_channels, kernel_size, batch_first, bias)
        self.MSE = nn.MSELoss(reduction = 'mean')
        self.training_hp = hyperparameters
        self.loss_train = []
        self.loss_test = []
        self.R_elc = torch.tensor([[]])   #
        self.R_prt = torch.tensor([[]])
        
        #self.save_hyperparameters()

    def training_step(self, batch, batch_idx):
        
        input = batch[0]     # Input of size (batch_size x timesteps_length x n_channels x height x width)
        targets = batch[1]   # Output of size (batch_size x 2).

        output = self.model(input)
        
        output = output[0:targets.shape[0],:]

        loss = self.MSE(output, targets)

        self.loss_train.append(loss.item())


        return loss

    def validation_step(self, batch, batch_idx):

        input = batch[0]     # Input of size (batch_size x timesteps_length x n_channels x height x width)
        targets = batch[1]   # Output of size (batch_size x 2).

        output = self.model(input)
        output = output[0:targets.shape[0],:]
        loss = self.MSE(output, targets)
        self.loss_test.append(loss.item())

        if self.R_prt.shape[1] != 0:
            cat_prt = torch.cat((output[:,0].unsqueeze(0), targets[:, 0].unsqueeze(0)), dim = 0)
            cat_elc = torch.cat((output[:,1].unsqueeze(0), targets[:, 1].unsqueeze(0)), dim = 0)

            self.R_prt = torch.cat((self.R_prt, cat_prt), dim = 1)
            self.R_elc = torch.cat((self.R_elc, cat_elc), dim = 1)
        else:
            self.R_prt = torch.cat((output[:,0].unsqueeze(0), targets[:, 0].unsqueeze(0)), dim = 0)
            self.R_elc = torch.cat((output[:,1].unsqueeze(0), targets[:, 1].unsqueeze(0)), dim = 0)


        return loss

    def configure_optimizers(self):
        
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr = self.training_hp['lr'], weight_decay = self.training_hp['wd'])

        return self.optimizer


In [17]:
s = CNN_LSTMmodule()
# t = torch.randn((12,4,1,224,224))
# out = s(t)

In [28]:
for i in SettingData.val_dataloader():
    
    out = s(i[0])
    break


In [29]:
i[0].shape

torch.Size([4, 9, 1, 224, 224])

In [30]:
torch.sum(out[1][0], dim =1)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [8]:
exp_name = '0.5_CNNLSTM_lr_e-5_wd_e-1'
load = False
save = False

path = './models/'+exp_name+'.pt'


if mod == 'conv':
    model = HeliosNet(n_channels, n_hidden_channels, kernel_size, n_layers, batch_first, bias, p_drop = training_hp['ConvLSTM_drop'])

elif mod == 'cnnlstm':
    model = CNN_LSTMmodule()

if load:
    
    load_model(path, model, device)
pl_training_MDL = TrainingModule(model, experiment_name = exp_name)




/home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.7 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [9]:
num_gpu = 1 if torch.cuda.is_available() else 0


if wb:
    # initialise the wandb logger and name your wandb project
    wandb_logger = WandbLogger(project=project_name, name = exp_name, config = hyperparameters, entity = 'small_sunbirds')

    trainer = pl.Trainer(
        max_epochs = hyperparameters['epochs'],  # maximum number of epochs.
        gpus=num_gpu,  # the number of gpus we have at our disposal.
        default_root_dir="", logger = wandb_logger, callbacks = [Get_Metrics()]
    )
    if save:
        trainer.callbacks.append(Save_Model_(path, exp_name))

else:
    trainer = pl.Trainer(
        max_epochs = hyperparameters['epochs'],  # maximum number of epochs.
        gpus=num_gpu,  # the number of gpus we have at our disposal.
        default_root_dir="", callbacks = [Get_Metrics()] 
    )
    if save:
        trainer.callbacks.append(Save_Model_(path, exp_name))
        

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [10]:
trainer.fit(model = pl_training_MDL, datamodule = SettingData)
wandb.finish()

/home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type           | Params
-----------------------------------------
0 | model | CNN_LSTMmodule | 7.9 M 
1 | MSE   | MSELoss        | 0     
-----------------------------------------
7.9 M     Trainable params
0         Non-trainable params
7.9 M     Total params
31.513    Total estimated model params size (MB)
/home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:631: UserWarning: Checkpoint directory /home/peppe/Desktop/Università/Projects/Solar_wind_density_forecasting/0.5_CNNLSTM_lr_e-5_wd_e-1/version_None/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint director

Validation sanity check: 0it [00:00, ?it/s]

/home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Global seed set to 312023
/home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Electron Density correlation (R),█▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
Loss on test,█▆▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂
Loss on train,█▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Proton Density correlation (R),█▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
trainer/global_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
Electron Density correlation (R),0.01507
Loss on test,17.09977
Loss on train,12.85415
Proton Density correlation (R),-0.02316
epoch,29


In [20]:
model.to(device)

HeliosNet(
  (ConvLSTM): ConvLSTM(
    (cell_list): ModuleList(
      (0): ConvLSTMCell(
        (conv): Conv2d(9, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (drop): Dropout(p=0.5, inplace=False)
      )
      (1): ConvLSTMCell(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (drop): Dropout(p=0.5, inplace=False)
      )
      (2): ConvLSTMCell(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (drop): Dropout(p=0.5, inplace=False)
      )
      (3): ConvLSTMCell(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (drop): Dropout(p=0.5, inplace=False)
      )
      (4): ConvLSTMCell(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (drop): Dropout(p=0.5, inplace=False)
      )
    )
  )
  (DensityPredictor): PredictionModule(
    (conv1): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (c

In [12]:
for i in SettingData.val_dataloader():
    i[1].to(device)
    i[0].to(device)

    print('real: \n',i[1])
    out = model(i[0])
    print('Predicted: \n', out)

real: 
 tensor([[0.7750, 0.9395],
        [1.0350, 1.0774],
        [1.6400, 1.6557],
        [2.4500, 1.9147]])
Predicted: 
 tensor([[5.3729, 3.8337],
        [5.3704, 3.8320],
        [5.3736, 3.8343],
        [5.3720, 3.8333]], device='cuda:0', grad_fn=<CopySlices>)
real: 
 tensor([[3.1450, 2.0417],
        [5.4450, 5.7560],
        [4.5400, 2.6724],
        [3.1050, 1.3998]])
Predicted: 
 tensor([[5.3745, 3.8350],
        [5.3773, 3.8373],
        [5.3770, 3.8370],
        [5.3821, 3.8403]], device='cuda:0', grad_fn=<CopySlices>)
real: 
 tensor([[ 8.2550,  6.7167],
        [ 8.8500,  5.5202],
        [19.9200, 14.8109],
        [ 5.7800,  4.4608]])
Predicted: 
 tensor([[5.3808, 3.8399],
        [5.3817, 3.8403],
        [5.3736, 3.8346],
        [5.3744, 3.8353]], device='cuda:0', grad_fn=<CopySlices>)
real: 
 tensor([[ 5.0400,  6.8342],
        [ 7.2400,  3.6822],
        [15.2550,  8.8776],
        [ 1.1850,  0.8767]])
Predicted: 
 tensor([[5.3773, 3.8372],
        [5.3767, 3.836

In [8]:
t = torch.randn((4, 5, 1, 224, 224))

# output = model(t)

In [13]:
output[0][0].shape

torch.Size([1, 8, 224, 224])

torch.Size([4, 2])


/home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
